# [R889] Investigate coupled channel fit on the channels $$ and $$ with the $F$ vector formalism in comparasion the the Flatte lineshape.
Genarate data with $F$ vector $F$ vector for n poles and n channels

In [1]:
%pip install -q ampform==0.15.0 sympy==1.12

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations

from dataclasses import dataclass

import ampform
import matplotlib.pyplot as plt
import numpy as np
import qrules
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import perform_cached_doit
from IPython.display import Math, display
from qrules.particle import Particle
from qrules.transition import ReactionInfo
from sympy.matrices.expressions.matexpr import MatrixElement
from tensorwaves.function.sympy import create_parametrized_function

from kmatrix import COLLECTED_X_SYMBOLS, create_dynamics_symbol

_ = np.seterr(invalid="ignore")

ModuleNotFoundError: No module named 'tensorwaves'

# Collect dynamics symbols

In [ ]:
FINAL_STATES: list[tuple[str, ...]] = [
    ["K0", "K~0", "gamma"],
    ["pi0", "pi0", "gamma"],
]
REACTIONS: list[ReactionInfo] = [
    qrules.generate_transitions(
        initial_state="J/psi(1S)",
        final_state=final_state,
        allowed_intermediate_particles=["f(0)(980)"],
        allowed_interaction_types=["strong", "em"],
        formalism="helicity",
        # particle_db=PARTICLE_DB,
        mass_conservation_factor=4.0,
    )
    for final_state in FINAL_STATES
]

In [ ]:
MODELS = []
for reaction in REACTIONS:
    builder = ampform.get_builder(reaction)
    builder.adapter.permutate_registered_topologies()
    builder.config.scalar_initial_state_mass = True
    builder.config.stable_final_state_ids = [0, 1, 2]
    for resonance in reaction.get_intermediate_particles():
        builder.set_dynamics(resonance.name, create_dynamics_symbol)
    MODELS.append(builder.formulate())

In [ ]:
selected_amplitudes = {
    k: v for i, (k, v) in enumerate(MODELS[0].amplitudes.items()) if i < 3
}

In [ ]:
for X, resonance_info in COLLECTED_X_SYMBOLS.items():
    for res, _L in sorted(resonance_info):
        display(X)
        print(f"  {res.name:<20s} {res.mass:>8g} GeV  {res.width:>8g} GeV")

In [ ]:
import kmatrix


@dataclass
class TwoBodyDecay:  # specific to the channel
    child1: Particle
    child2: Particle


DECAYS = tuple(
    TwoBodyDecay(
        child1=reaction.final_state[0],
        child2=reaction.final_state[1],
    )
    for reaction in REACTIONS
)
s = sp.Symbol("m_01", real=True) ** 2

PARAMETERS = {}
for model in MODELS:
    PARAMETERS.update(model.parameter_defaults)
    del model

resonances, *_ = COLLECTED_X_SYMBOLS.values()

## Formulate dynamics expression

In [ ]:
for symbol, resonances in COLLECTED_X_SYMBOLS.items():
    display(symbol)
    for p, _ in resonances:
        print(f"  {p.name:<20s} {p.mass:>8g} GeV  {p.width:>8g} GeV ")
MODELS[0].parameter_defaults

## Formulate Dynamics

### Define matrix symbols

In [ ]:
n_channels = len(REACTIONS)
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
P = sp.MatrixSymbol("P", n_channels, 1)
T = sp.MatrixSymbol("T", n_channels, n_channels)
F = sp.MatrixSymbol("F", n_channels, 1)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)
rho_cm = sp.MatrixSymbol(R"{\rho_{\Sigma}}", n_channels, n_channels)



### $K$ matrix 

As used in the PDG resonance section a different form of the $K$ matrix is used containing the the $g^{0}$ couplings instet of the unitless $\gamma$ constants.

In [ ]:
def formulate_K_matrix(
    resonances: list[tuple[Resonances, int]], n_channels: int
) -> dict[MatrixElement, sp.Expr]:
    Kmatrix_expressions = {}
    for i in range(n_channels):
        for j in range(n_channels):
            resonance_contributions = []
            for res, _L in resonances:
                s = sp.Symbol("m_01", real=True) ** 2
                g_Ri = sp.Symbol(Rf"g_{{{res.latex},{i}}}")
                g_Rj = sp.Symbol(Rf"g_{{{res.latex},{j}}}")
                m_R = sp.Symbol(Rf"m_{{{res.latex}}}")
                parameter_defaults = {
                    m_R: res.mass,
                    g_Ri: 1,
                    g_Rj: 1,
                }
                PARAMETERS.update(parameter_defaults)
                expr = (g_Ri * g_Rj) / (m_R**2 - s)
                resonance_contributions.append(expr)
            Kmatrix_expressions[K[i, j]] = sum(resonance_contributions)

    return Kmatrix_expressions


K_expressions = formulate_K_matrix(resonances, n_channels=len(REACTIONS))
Math(aslatex(K_expressions))
K_matrix = K.as_explicit()
K.as_explicit().xreplace(K_expressions)

### $P$ vector

In [ ]:
def formulate_P_vector(
    resonances: list[tuple[Resonances, int]], n_channels: int
) -> dict[MatrixElement, sp.Expr]:
    P_expressions = {}
    for i in range(n_channels):
        resonance_contributions = []
        for res, _L in resonances:
            s = sp.Symbol("m_01", real=True) ** 2
            beta_R = sp.Symbol(Rf"\beta_{{{res.latex}}}")
            m_R = sp.Symbol(Rf"m_{{{res.latex}}}")
            g_Ri = sp.Symbol(Rf"g_{{{res.latex},{i}}}")
            parameter_defaults = {
                beta_R: 1 + 0j,
            }
            PARAMETERS.update(parameter_defaults)
            expr = (beta_R * g_Ri) / (m_R**2 - s)
            resonance_contributions.append(expr)
        P_expressions[P[i, 0]] = sum(resonance_contributions)

    return P_expressions


P_expressions = formulate_P_vector(resonances, n_channels=len(REACTIONS))
P_vector = P.as_explicit()
Math(aslatex(P_expressions))

### Phase space with Chew Mandelstam function

In [ ]:
def formulate_PhspCM_matrix(n_channels: int) -> dict[sp.MatrixElement, sp.Expr]:
    Phsp_matrix_CM_expressions = {}

    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                s = sp.Symbol("m_01", real=True) ** 2
                rho_i = kmatrix.PhaseSpaceCM(s, m_a_i, m_b_i)
                Phsp_matrix_CM_expressions[rho_cm[i, j]] = rho_i
                parameter_defaults = {
                    m_a_i: DECAYS[i].child1.mass,
                    m_b_i: DECAYS[i].child2.mass,
                }
                PARAMETERS.update(parameter_defaults)
            else:
                Phsp_matrix_CM_expressions[rho_cm[i, j]] = 0

    return Phsp_matrix_CM_expressions


Phsp_CM_expressions = formulate_PhspCM_matrix(n_channels=len(REACTIONS))
rho_cm.as_explicit().xreplace(Phsp_CM_expressions)
PHSP_CM_expressions = np.array([
    perform_cached_doit(rho_cm[i, i].xreplace(Phsp_CM_expressions))
    for i in range(n_channels)
])
Math(aslatex(PHSP_CM_expressions))

### $F$ vector

:::{note}
For some reason one has to leave out the multiplication of $\rho$ by $i$ within the calculation of the $F$ vector
:::

In [ ]:
F = (I - sp.I * K * rho_cm).inv() * P
F

In [ ]:
F_vector = F.as_explicit()
F_vector

In [ ]:
combined_expressions_F = {**K_expressions, **Phsp_CM_expressions, **P_expressions}

In [ ]:
Math(aslatex(e.xreplace(combined_expressions_F).simplify(doit=False) for e in F_vector))

In [ ]:
F_expressions = np.array([
    perform_cached_doit(F_vector[i].xreplace(combined_expressions_F))
    for i in range(n_channels)
])
F_expressions[1]

### Plot lineshape of $F$ vector 

Coupling parameters $g_{KK}$ and $g_{\pi\pi}$taken from [Phys.Atom.Nucl.65:1545-1552,2002](https://arxiv.org/pdf/hep-ph/0102338.pdf)

In [ ]:
new_parameters_fvector = {
    sp.Symbol(R"g_{f_{0}(980),0}"): np.sqrt(0.186),
    sp.Symbol(R"g_{f_{0}(980),1}"): -np.sqrt(0.076),
    sp.Symbol(R"m_{f_{0}(980)}"): 0.98,
}
PARAMETERS.update(new_parameters_fvector)
PARAMETERS

In [ ]:
LINESHAPE = []
PHSP = []
for i in range(n_channels):
    LINESHAPE.append(
        create_parametrized_function(
            expression=F_expressions[i],
            backend="jax",
            parameters=PARAMETERS,
        )
    )
for i in range(n_channels):
    PHSP.append(
        create_parametrized_function(
            expression=PHSP_CM_expressions[i],
            backend="jax",
            parameters=PARAMETERS,
        )
    )
epsilon = 1e-7j
x = np.linspace(0.7, 1.5, num=300)
data1 = {"m_01": x + epsilon}
for i in range(n_channels):
    y_values = (np.abs(LINESHAPE[i](data1)) ** 2 * PHSP[i](data1)).real
    plt.plot(x, y_values, label=f"Channel {i + 1}")
plt.ylabel(R"$F$ vector")
plt.xlabel("s [MeV]")
plt.title("Lineshapes")
plt.legend()
plt.show()

## $T$ matrix

In [ ]:
T = (I - sp.I * K * rho_cm).inv() * K
T

In [ ]:
T_matrix = T.as_explicit()
T_matrix

In [ ]:
combined_expressions_T = {**K_expressions, **Phsp_CM_expressions}

In [ ]:
Math(aslatex(e.xreplace(combined_expressions_T).simplify(doit=False) for e in T_matrix))

### Calculate unphysical sheets

In [ ]:
def formulate_Phsprho_matrix(n_channels: int) -> dict[sp.MatrixElement, sp.Expr]:
    Phsp_matrix_rho_expressions = {}

    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                s = sp.Symbol("m_01", real=True) ** 2
                rho_i = kmatrix.PhaseSpaceCM(s, m_a_i, m_b_i)
                Phsp_matrix_rho_expressions[rho[i, j]] = rho_i
                parameter_defaults = {
                    m_a_i: DECAYS[i].child1.mass,
                    m_b_i: DECAYS[i].child2.mass,
                }
                PARAMETERS.update(parameter_defaults)
            else:
                Phsp_matrix_rho_expressions[rho[i, j]] = 0

    return Phsp_matrix_rho_expressions


Phsp_rho_expressions = formulate_Phsprho_matrix(n_channels=len(REACTIONS))
rho.as_explicit().xreplace(Phsp_rho_expressions)

In [ ]:
T_II = (T.inv() - 2 * sp.I * rho).inv()
#T_III = (T.inv() + 2 * sp.I * rho).inv()
#T_IV = (-T.inv() - 2 * sp.I * rho).inv()

In [ ]:
T_II_expl = T_II.as_explicit()
#T_III_expl = T_III.as_explicit()
#T_IV_expl = T_IV.as_explicit()

In [ ]:
combined_expressions_T_unpys = {
    **K_expressions,
    **Phsp_CM_expressions,
    **Phsp_rho_expressions,
}

In [ ]:
T_II_expl.xreplace(combined_expressions_T_unpys)
#T_III_expl.xreplace(combined_expressions_T_unpys)
#T_IV_expl.xreplace(combined_expressions_T_unpys)

## Calculate $F$ vector frm $T$ matrix

In [ ]:
F_I = T * K.inv() * P
F_II = T_II * K.inv() * P
#F_III = T_III * K.inv() * P
#F_IV = T_IV * K.inv() * P
F_II[0]

In [ ]:
F_I_expl = F_I.as_explicit()
F_II_expl = F_II.as_explicit()
#F_III_expl = F_III.as_explicit()
#F_IV_expl = F_IV.as_explicit()
F_II_expl

In [ ]:
combined_expressions_F_unpys = {
    **K_expressions,
    **Phsp_CM_expressions,
    **Phsp_rho_expressions,
    **P_expressions,
}

In [ ]:
Math(
    aslatex(
        e.xreplace(combined_expressions_F_unpys).simplify(doit=False) for e in F_II_expl
    )
)

In [ ]:
F_I_expressions = np.array([
    perform_cached_doit(F_I_expl[i].xreplace(combined_expressions_F_unpys))
    for i in range(n_channels)
])

F_II_expressions = np.array([
    perform_cached_doit(F_II_expl[i].xreplace(combined_expressions_F_unpys))
    for i in range(n_channels)
])
F_II_expressions[0]

In [ ]:
LINESHAPE_FI = []
PHSP_FI = []
for i in range(n_channels):
    LINESHAPE_FI.append(
        create_parametrized_function(
            expression=F_I_expressions[i],
            backend="jax",
            parameters=PARAMETERS,
        )
    )
LINESHAPE_FII = []
PHSP_FII = []
for i in range(n_channels):
    LINESHAPE_FII.append(
        create_parametrized_function(
            expression=F_II_expressions[i],
            backend="jax",
            parameters=PARAMETERS,
        )
    )
for i in range(n_channels):
    PHSP_FI.append(
        create_parametrized_function(
            expression=PHSP_CM_expressions[i],
            backend="jax",
            parameters=PARAMETERS,
        )
    )
epsilon = 1e-7j
x = np.linspace(0.7, 1.5, num=300)
data1 = {"m_01": x + epsilon}
for i in range(n_channels):
    y_values = (np.abs(LINESHAPE_FI[i](data1)) ** 2 * PHSP_FI[i](data1)).real
    plt.plot(x, y_values, label=f"Channel {i + 1}")
plt.ylabel(R"$F$ vector")
plt.xlabel("s [MeV]")
plt.title("Lineshapes")
plt.legend()
plt.show()

In [ ]:
project = np.imag
FP_II = []
FN_II = []
FP_I = []
FN_I = []
epsi = 1e-10
y = np.linspace(epsi, 1, num=100)
X, Y = np.meshgrid(x, y)
data_p = {"m_01": X + Y * 1j}
data_n = {"m_01": X - Y * 1j}
for i in range(n_channels):
    FP_I.append(LINESHAPE_FI[i](data_p))
    FN_I.append(LINESHAPE_FI[i](data_n))
    FP_II.append(LINESHAPE_FII[i](data_p))
    FN_II.append(LINESHAPE_FII[i](data_n))

In [ ]:
%config InlineBackend.figure_formats = ["png"]
for i in range(n_channels):
    fig, ax = plt.subplots(figsize=(12, 8), sharey=True)
    ax.set_xlabel(R"$\mathrm{Re}\,s$")
    ax.set_ylabel(R"$\mathrm{Im}\,s$")

    F_max = 1
    style = dict(vmin=-F_max, vmax=+F_max, cmap=plt.cm.coolwarm)
    mesh = ax.pcolormesh(X, Y, FP[i].imag, **style)
    mesh = ax.pcolormesh(X, -Y, -FN[i].imag, **style)
    plt.subplots_adjust(wspace=1)
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    plt.colorbar(mesh, cax=cbar_ax)

    fig.tight_layout(rect=[0, 0.03, 0.9, 0.95])

    plt.show()